In [1]:
#Initial environement of computations

import matplotlib.pyplot as plt
import numpy as np
from kanon.calendars import Calendar, Date
from kanon.calendars.calendars import float_to_hm, hm_to_float
from kanon.tables import HTable
from kanon.units import Sexagesimal, Historical
from kanon.units.precision import set_precision, TruncatureMode, PrecisionMode
import astropy.units as u


from kanon.tables.htable_reader import (
    DISHAS_REQUEST_URL,
    read_historical,
    read_sexag_array,
    read_intsexag_array,
)
from kanon.tables import HTable
import requests


number_reader = {
    "sexagesimal": read_sexag_array,
    "floating sexagesimal": read_sexag_array,
    "integer and sexagesimal": read_intsexag_array,
    "historical": read_historical,
}

In [2]:
#Initial table set

# Historical target table
Hist_table=HTable.read(1405, with_units=False)
Hist_table


hours,Entries
IntegerAndSexagesimal,Historical
1 ;,"00 ; 07,46"
2 ;,"00 ; 15,32"
3 ;,"00 ; 23,18"
4 ;,"00 ; 31,04"
5 ;,"00 ; 30,15"
6 ;,"00 ; 46,36"
7 ;,"00 ; 54,22"
8 ;,"01 ; 02,08"
9 ;,"01 ; 09,54"


In [3]:
# Formating and initialising the Test tables

Test_recomp=Hist_table.copy()
Delta_recomp=Hist_table.copy()
for i in range(30):
    Test_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)
    Delta_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)


In [4]:
# mean motion parameter from the Parisian Alfonsine tables
daily_mean=Historical(Sexagesimal("3;6,24,7,42,40,52"),7)
REVO=Historical(Sexagesimal("6,0;0,0"),7)

for i in range(30):
    with set_precision(tmode=TruncatureMode.TRUNC, pmode=2):
        Test_recomp[i][1]=daily_mean*((i+1)/24)%REVO
        with set_precision(tmode=TruncatureMode.ROUND, pmode=2):
            Test_recomp[i][1]=1*Test_recomp[i][1]
            Delta_recomp[i][1]=3600*(Hist_table[i][1]-Test_recomp[i][1])

In [5]:
# residuals visualisation and statistical summaries 1
B=Delta_recomp.to_pandas()
(B.mean().mean(), B.std().std())

(-35.5, nan)

In [6]:
B.astype(float)

,Entries
hours,
1 ;,0.0
2 ;,0.0
3 ;,0.0
4 ;,0.0
5 ;,-515.0
6 ;,0.0
7 ;,0.0
8 ;,0.0
9 ;,0.0


In [7]:
#csv export of the recomputed table
B=Test_recomp.to_pandas()
compression_opts = dict(method='zip',archive_name='Mercury mean hours.csv')
B.to_csv('Mercury mean hours.zip', index=False, compression=compression_opts)